In [ ]:
import requests
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import  KFold
import warnings
import re
warnings.filterwarnings('ignore')

In [ ]:
api_key = "a7a0813f3d0ae3a4c26bf99545474eae"
api_base_url = "https://api.themoviedb.org/3"

discover_endpoint = f"{api_base_url}/discover/movie?api_key={api_key}&with_original_language=en"  

movie_endpoint = f"{api_base_url}/movie/"

def create_discover_endpoint(i,sort):
  return f"{discover_endpoint}&page={i}&sort_by={sort}"

def create_movie_endpoint(movie_id):
  return f"{movie_endpoint}{movie_id}?api_key={api_key}"

def create_credits_endpoint(movie_id):
  return f"{movie_endpoint}{movie_id}/credits?api_key={api_key}"


In [ ]:
requests.get(create_movie_endpoint(8967))

Récupération et sauvegarde des ids de films

In [ ]:
ids=[]

movie_sorting = ["popularity.asc", "popularity.desc", "release_date.asc", "release_date.desc", "revenue.asc", "revenue.desc", "primary_release_date.asc", "primary_release_date.desc", "original_title.asc", "original_title.desc", "vote_average.asc", "vote_average.desc", "vote_count.asc", "vote_count.desc"]

for sort in movie_sorting :
  for i in range(1,501):
    r=requests.get(create_discover_endpoint(i,sort))
    for j in range(len(r.json()["results"])): 
      try:
        if r.json()["results"][j]["release_date"] != '' :
          ids.append(r.json()["results"][j]["id"])
      except :
        pass

In [ ]:
ids = set(ids)
len(ids)

In [ ]:
with open('movies_ids.csv', 'w') as f:
    write = csv.writer(f)  
    write.writerow(list(ids))

Import des ids de films

In [ ]:
with open('movies_ids.csv', 'r') as f:
    ids = f.readlines()

ids = ids[0].split(',')
ids = [int(x) for x in ids]

In [ ]:
print(len(ids))

Récupération des infos sur les films

In [ ]:
movies_db_columns=['id','title','budget','genres','original_language','overview','production_companies','release_date','revenue','runtime','vote_average','vote_count','actors','director','producers']
movies = []
pestiferes = []

for id in ids:
  print(id)
  r=requests.get(create_movie_endpoint(id))
  movie = r.json()
  
  try:
    if movie['budget']!=0 and movie['revenue']!=0:
      title = movie['title']
      
      budget = movie['budget']
      revenue = movie['revenue']
      genres = [movie['genres'][i].get('name') for i in range(len(movie['genres']))]
      original_language = movie['original_language']
      overview = movie['overview']
      production_companies = [movie['production_companies'][i].get('name') for i in range(len(movie['production_companies']))]
      release_date = movie['release_date']
      
      runtime = movie['runtime']
      vote_average = movie['vote_average']
      vote_count = movie['vote_count']

      r = requests.get(create_credits_endpoint(id))
      cast = r.json()['cast']
      try:
        actors = [cast[i] for i in range(5)]
      except IndexError:
        actors = [cast[i] for i in range(len(cast))]

      crew = r.json()['crew']

      directors = [crew[i]for i in range(len(crew)) if crew[i].get('job') == 'Director']
      producers = [crew[i] for i in range(len(crew)) if crew[i].get('job') == 'Producer']
      
      movies.append([id,title,budget,genres,original_language,overview,production_companies,release_date,revenue,runtime,vote_average,vote_count,actors,directors,producers])
    else:
      pestiferes.append([id,movie['title'],movie['budget'],movie['revenue']])
  except KeyError as e:
    print(e, id)



In [ ]:
movies_db = pd.DataFrame(movies,columns=movies_db_columns)
movies_db.info()

Export CSV

In [ ]:
movies_db.to_csv("movies_db_1.csv",index=False)